# Cell Correction

This part shows how to correct cells in Stereopy. Generally, there are two ways to complete it:

    correcting from BGEF and mask;

    correcting from GEM and mask.

<div class="alert alert-info">

**Preparation**

Before **Cell Segmentation**, **Tissue Segmentation** or generating **mask file** from **ssDNA image** on **Cell Correction**, certain packages should be installed beforehand.  

Install necessary packages by running commands as below:

> &ensp;&ensp; **pip install tensorflow==2.7.0**  
> &ensp;&ensp; **pip install torch==1.10.0**  
> &ensp;&ensp; **pip install torchvision==0.11.1**  
> &ensp;&ensp; **pip install albumentations==0.4.6**

</div>

Provided with three algorithmic methods, you could determine which version to perform by setting the parameter `fast`, including `False`, `v1` and `v2`.

1. `False`: slower but more accurate, it is based on GMM and will uses multiprocessing if set `process_count` to more than 1.
2. `v1`: fast but less accurate, only supports single process and single threading.
3. `v2`: default and recommended algorithm, the latest fast version which is the most accurate and faster than `v1` uses multithreading if set `process_count` to more than 1.

More details refer to [**API**](https://stereopy.readthedocs.io/en/latest/api/stereo.tools.cell_correct.cell_correct.html#stereo.tools.cell_correct.cell_correct).

`v2` algorithm is expected to eliminate the overlapping influence as much as possible, you can obvisouly learn from the following image that there is almost no overlap after cell correcting.

![cell_correction_effect.png](https://raw.githubusercontent.com/BGIResearch/stereopy/main/docs/source/_static/cell_correction_effect.png)

## Correcting from BGEF and Mask

On this way, you should specify the path of BGEF by `bgef_path`, the path of mask by `mask_path` and the output path to save corrected result by `out_dir`. 

Cell correction dafaults to return a StereoExpData object, if set `only_save_result` to `True`, only return the path of CGEF after correcting.

If you have no mask currently, you can generate it from ssDNA image, refer to [**Cell Segmentation**](https://stereopy.readthedocs.io/en/latest/Tutorials/Cell_Segmentation.html).

In [ ]:
from stereo.tools.cell_correct import cell_correct

bgef_path = "SS200000135TL_D1.raw.gef"
mask_path = "SS200000135TL_D1_mask.tif"
out_dir = "cell_correct_result"

data = cell_correct(
                    out_dir=out_dir,
                    bgef_path=bgef_path,
                    mask_path=mask_path,
                    only_save_result=False,
                    fast='v2'
                    )

Output directory includes such files:

1. `.raw.cellbin.gef` - the CGEF without correcting, generated from BGEF and mask;
2. `.adjusted.gem` - the gem after correction;
3. `.adjusted.cellbin.gef` - the CGEF after correcting, generated from the `.adjusted.gem`;
4. `err.log` - records the cells which cannot be corrected and not contained in `.adjusted.gem` and `.adjusted.cellbin.gef`.

## Correcting from GEM and Mask

In this way, you should also specify the path of BGEF by `bgef_path`, the path of mask by `mask_path` and the output path to save corrected result by `out_dir`. 

In output directory, the file named `*.bgef` is generated form mask.

In [ ]:
from stereo.tools.cell_correct import cell_correct

gem_path = "SS200000135TL_D1.cellbin.gem"
mask_path = "SS200000135TL_D1_mask.tif"
out_dir = "cell_correct_result"

data = cell_correct(
                    out_dir=out_dir,
                    gem_path=gem_path,
                    mask_path=mask_path,
                    only_save_result=False,
                    fast='v2'
                    )

## Running on Jupyter Notebook

Notebook can not support multiprocess directly, we recommend following two steps to improve performance.

Firstly, write the source code in a python file by command `%%writefile`.

In [ ]:
%%writefile temp.py
from stereo.tools.cell_correct import cell_correct

bgef_path = "SS200000135TL_D1.raw.gef"
mask_path = "SS200000135TL_D1_mask.tif"
out_dir = "cell_correct_result"

data = cell_correct(
                    out_dir=out_dir,
                    bgef_path=bgef_path,
                    mask_path=mask_path,
                    process_count=10,
                    only_save_result=False,
                    fast=False
                    )

Secondly, run the file by command `%run`:

In [ ]:
%run temp.py

<div class="alert alert-info">

**Note**

We strongly suggest you output in CGEF format for subsequent analysis. When you transform CGEF to CGEM for intuitive understantding, there will be a small quantity of genes which are lost because of the algorithm, related to cell boarder. If you are concerned about the lost, just make a comparasion on the gene expression amount of the missing part.

</div>

## Performance

Take a GEF containing 55460 cells and 25546 genes as an example.

Machine configuration as below:

|physical cores |logic cores |memory   |
| --- | --- | --- |
|12             |48          |250G     |

Comparision of performance:

`fast=False`

|process  |memory(max) |cpu    |time   |
| --- | --- | --- | --- |
|10       |140G        |2330%  |2h13m  |

`fast=v1` (only support single process)

|process  |memory(max) |cpu    |time   |
| --- | --- | --- | --- |
|1        |49G         |99%    |40m    |

`fast=v2` (single process)

|process  |memory(max) |cpu    |time   |
| --- | --- | --- | --- |
|1        |50G         |100%    |23m    |